# Google Drive

In [2]:
####################################
#
#  ADD THIS TO EVERY COLAB FILE!
#
####################################

from google.colab import drive
drive.mount('/content/drive')

import drive.Shareddrives.GPTJ.project.settings as settings

PATH_PROJECT = settings.PATH_PROJECT
PATH_DATA = settings.PATH_DATA

Mounted at /content/drive


In [3]:
! cd $PATH_PROJECT && pip install -q -r requirements.txt

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 143 kB 5.3 MB/s 
     |████████████████████████████████| 55.9 MB 268 kB/s 
     |████████████████████████████████| 85 kB 4.6 MB/s 
     |████████████████████████████████| 72 kB 1.3 MB/s 
     |████████████████████████████████| 163 kB 72.9 MB/s 
     |████████████████████████████████| 7.6 MB 37.0 MB/s 
     |████████████████████████████████| 1.3 MB 48.4 MB/s 
     |████████████████████████████████| 441 kB 51.0 MB/s 
     |████████████████████████████████| 212 kB 60.3 MB/s 
     |████████████████████████████████| 115 kB 71.7 MB/s 
     |████████████████████████████████| 127 kB 72.7 MB/s 
     |████████████████████████████████| 115 kB 71.6 MB/s 


In [4]:
!nvidia-smi

# Standard -> Tesla T4
# Premium -> Tesla P100-PCIE-16GB

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [5]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel, GPTNeoForCausalLM

torch.manual_seed(42)

# Dataset

In [6]:

class SimpleDataset(Dataset):

    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []

        for txt in txt_list:
            # Encode the descriptions using the GPT-Neo tokenizer
            encodings_dict = tokenizer('<|startoftext|>' 
                                        + txt +    
                                        '<|endoftext|>',
                                        truncation=True,
                                        max_length=max_length, 
                                        padding='max_length')
            input_ids = torch.tensor(encodings_dict['input_ids'])    
            self.input_ids.append(input_ids)
            mask = torch.tensor(encodings_dict['attention_mask'])
            self.attn_masks.append(mask)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]


# Treino

In [7]:
model_name = 'gpt2'

tokenizer = GPT2Tokenizer.from_pretrained(model_name, 
    bos_token='<|startoftext|>',
    eos_token='<|endoftext|>', 
    pad_token='<|pad|>'
)
model = GPT2LMHeadModel.from_pretrained(model_name) #.cuda()
model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Embedding(50259, 768)

In [8]:
path = '/content/drive/Shareddrives/GPTJ/data/kaggle/proc-1/5/'

descriptions = []
listdir = list(os.listdir(path))[:20]
for filename in listdir:
    filepath = os.path.join(path, filename)
    with open(filepath, 'r') as f:
        descriptions.append(f.read()[:1000])

print('len desc', len(descriptions))
max_length = max([len(tokenizer.encode(description)) for description in descriptions])

dataset = SimpleDataset(descriptions, tokenizer, max_length)

print(len(dataset))

train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

len desc 20
20


In [9]:


training_args = TrainingArguments(
    output_dir='/content/',
    num_train_epochs=1,
    # logging_steps=5000,
    # save_steps=5000,                                   
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=0,
    weight_decay=0.01,  
    # logging_dir=os.path.join(PATH_DATA, model_name, 'logs')
)

trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset, 
    # This custom collate function is necessary 
    # to built batches of data
    data_collator=lambda data: {
        'input_ids': torch.stack([f[0] for f in data]),       
        'attention_mask': torch.stack([f[1] for f in data]),
        'labels': torch.stack([f[0] for f in data])
    }
)
# Start training process!
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 18
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 9


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=9, training_loss=21.201158311631943, metrics={'train_runtime': 411.6426, 'train_samples_per_second': 0.044, 'train_steps_per_second': 0.022, 'total_flos': 4124535552000.0, 'train_loss': 21.201158311631943, 'epoch': 1.0})

# Complete

In [ ]:
prompt = 'mov ebx 1000h'
generated = tokenizer(prompt, return_tensors='pt').input_ids  #.cuda()

sample_outputs = model.generate(generated, 
    # Use sampling instead of greedy decoding 
    do_sample=True, 
    top_k=50, 
    max_length=50,
    top_p=0.95,
    temperature=0.001,              
    num_return_sequences=5
)

# Print generated descriptions
for i, sample_output in enumerate(sample_outputs): 
    print('{}: {}'.format(i, tokenizer.decode(sample_output, skip_special_tokens=True)).replace('\n', ' '))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
